# 모델 고도화

- Few-shot prompting
- Chain-of-thought prompting

In [1]:
import math
import re

from tqdm.notebook import tqdm

from eval import get_eval_data, pointwise_eval
from utils import summarize, get_train_data

In [2]:
PROMPT_BASELINE = f"""아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

## Get train Data

In [27]:
conv_train = get_train_data()[18]
print(conv_train)

P01: 아프간은 지금 식량난이 엄청 심하다네 ㅠ
P02: 에휴...
P02: 거기는  뭘 해도 살기 힘든 나라인 거 같아...ㅠ ㅠ
P01: 탈레반 점령 전부터 그러기 시작했는데 밀이 주식인데 25프로? 그 정도나 올라서 못 먹는다나 봐 ㅜ
P02: 아이고야...
P02: 진짜 그 나라는 어찌 돌아 갈라구 서민들 식량을 저렇게나 올리는 거야...
P02: 저건 죽으라는 거랑 똑같은 거잖아 진짜 너무 잔인한 거 같아...ㅠ ㅠ
P01: 그게 전쟁도 너무 오랫동안 지속되고 가뭄도 심해서 농작물 피해도 어마어마한가 봐 ㅠ
P01: 그래서 식량 조달도 어렵고 그래서 그렇게 됐나 봐 ㅠ
P02: 그렇지 ㅠ ㅠ...
P02: 난 그래도 우리나라에서 태어나서 정말 다행이다 생각해...
P02: 저런 나라에서 태어나서 자라면 너무 열악한 환경에서 지내게 될 꺼 같아...
P02: 생각만 해도 답답 ㅠㅠ
P01: 우리나라도 살면서 그리 좋은 느낌은 없지만ㅠ
P01: 이런 나라 이야기를 들으면 현재에 감사하게 되는 것 같아 ㅠ
P02: 맞아...
P02: 그래도 인권이 존중될 수 있는 나라니까...
P02: 100% 마음에는 안 들지만 내가 이 나라에서 내 일을 하면서 아이를 잘 키울 수 있겠다란 생각은 들더라구...
P02: 저런 곳에서 어떻게 애를 키우나 그 생각부터 들었어 ㅜ ㅜ
P01: 저기선 절대 아이 안 낳고 싶을 듯 ㅜ
P01: 그런데 그런 자유도 없겠지? ㅠ
P02: 그치
P02: 결혼도 내가 안하고 싶다고 안 할 수 없으니 강제로 결혼 시키잖아 몇 살 되면 무조건~
P02: 여자의 인권을 아예 무시하는 나라야 정말 살기 싫어 ㅜㅜ...
P01: 그러게 아 얼마 전에 벨기에 대사관 부인 사건 있었잖아~
P01: 그 사람은 다시 고국으로 돌아갔나 봐
P01: 새로운 벨기에 대사관이 오는데, 부인이 한국인이라나 봐~
P02: 진짜? 그분 대단하네 어떻게 대사관 부인이 되셨대! ^^
P01: 글쎄 키키 그건 모르겠는데 예전에도 대사관으로 와서 있었는데 그때 만났나 하

In [31]:
import keyring
import anthropic

client = anthropic.Anthropic(
    api_key=keyring.get_password('anthropic', 'key_for_windows')
)

prompt = f"당신은 요약 전문가입니다. 사용자 대화들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 사용자 대화를 3문장 내로 요약해주세요: {conv_train}"

message = client.messages.create(
    model='claude-3-5-sonnet-20240620',
    max_tokens=1000,
    temperature=0.0,
    messages=[
        {'role': 'user', 'content': prompt}
    ]
)

message.content[0].text

'이 대화는 아프가니스탄의 심각한 식량난과 열악한 생활 환경에 대해 논의하고 있습니다. 두 사람은 한국에서 태어난 것에 감사함을 표현하며, 아프가니스탄과 같은 나라에서의 삶의 어려움을 공감합니다. 대화 후반부에는 벨기에 대사관 부인 관련 사건과 새로운 한국인 대사관 부인에 대한 이야기로 주제가 전환됩니다.'

In [32]:
summary_sample = message.content[0].text

prompt = f"""당신은 요약 전문가입니다. 사용자 대화들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 사용자 대화와 요약 예시입니다.
예시 대화:
{conv_train}
예시 요약 결과:
{summary_sample}
    
아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

In [33]:
models = [
    'claude-3-haiku-20240307',
    'gemini-1.5-flash-001',
    'gpt-3.5-turbo-0125'
]

scores = {model: [] for model in models}
pattern = r'\[\[\d+\]\]'

for model in models:
    for i in tqdm(range(5)):
        summary = summarize(
            conversation=get_eval_data()[i],
            prompt=prompt,
            model=model
        )
        eval_comment = pointwise_eval(get_eval_data()[i], summary)
        match = re.search(pattern, eval_comment)
        matched_string = match.group(0)
        score = int(matched_string[2])
        scores[model].append(score)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [34]:
for model in scores:
    print(scores[model], model)

[9, 8, 7, 9, 6] claude-3-haiku-20240307
[9, 8, 8, 9, 7] gemini-1.5-flash-001
[8, 8, 6, 7, 7] gpt-3.5-turbo-0125


## Chain-of-thought

In [35]:
prompt = f"""당신은 요약 전문가입니다. 사용자 대화들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 대화를 요약할 때는 다음 단계를 따라주세요:

1. 대화 참여자 파악: 대화에 참여하는 사람들의 수와 관계를 파악합니다.
2. 주제 식별: 대화의 주요 주제와 부차적인 주제들을 식별합니다.
3. 핵심 내용 추출: 각 주제에 대한 중요한 정보나 의견을 추출합니다.
4. 감정과 태도 분석: 대화 참여자들의 감정이나 태도를 파악합니다.
5. 맥락 이해: 대화의 전반적인 맥락과 배경을 이해합니다.
6. 특이사항 기록: 대화 중 특별히 눈에 띄는 점이나 중요한 사건을 기록합니다.
7. 요약문 작성: 위의 단계에서 얻은 정보를 바탕으로 간결하고 명확한 요약문을 작성합니다.
각 단계를 수행한 후, 최종적으로 전체 대화를 200자 내외로 요약해주세요.

예시 대화:
{conv_train}
예시 요약 결과:
{summary_sample}
    
아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

In [36]:
models = [
    'claude-3-haiku-20240307',
    'gemini-1.5-flash-001',
    'gpt-3.5-turbo-0125'
]

scores = {model: [] for model in models}
pattern = r'\[\[\d+\]\]'

for model in models:
    for i in tqdm(range(5)):
        summary = summarize(
            conversation=get_eval_data()[i],
            prompt=prompt,
            model=model
        )
        eval_comment = pointwise_eval(get_eval_data()[i], summary)
        match = re.search(pattern, eval_comment)
        matched_string = match.group(0)
        score = int(matched_string[2])
        scores[model].append(score)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [37]:
for model in scores:
    print(scores[model], model)

[8, 8, 6, 9, 6] claude-3-haiku-20240307
[9, 9, 8, 9, 1] gemini-1.5-flash-001
[7, 6, 6, 7, 6] gpt-3.5-turbo-0125


In [38]:
import math

for model in scores:
    mean = sum(scores[model]) / len(scores[model])
    variance = sum((x - mean) ** 2 for x in scores[model]) / (len(scores[model]) - 1)
    std_dev = math.sqrt(variance)
    print(f'{model}: {mean} / {round(std_dev, 2)}')

claude-3-haiku-20240307: 7.4 / 1.34
gemini-1.5-flash-001: 7.2 / 3.49
gpt-3.5-turbo-0125: 6.4 / 0.55


In [39]:
print(get_eval_data()[10])

P01: 토스에 아이디어스 떴어
P02: 아직도 만원 되나 떠있는데 엄마 것도 할라니 조카 등원 시키주는 중이네 키키
P02: 만원이가?
P02: 난 신랑 거 지난 주 떠서 했어
P02: 그거삿어 미아 방지 목걸이 무배
P02: 12900원짜리 사고 9천원 돌려받고
P02: 막창도 많이 사더라
P01: 엄마 거도 되나?
P01: 근데 울집 안 돼서 머 키키
P01: 비티비 썩었네
P02: 업뎃되고 안 된다고 댓글 있더라
P01: 긍게   됐다 키키 난 이만 하믄 키키
P02: 아이디어스는 만원짜리가?
P01: 응 70프로만원
P02: 사라 키키 보고 키키 혹시 막창 살 거면 키키 내장 좀 하나 사주고 키키
P01: 내 손수건 사야 된다. .
P01: 난**때문에 수명 단축될 듯...준비 하나도 안했는데 학교에 잠바 갖다주러 갔다옴...
P01: 손 닦는 수건은 거칠하다가코 아기 상어 그려져 있어서 부끄럽다 카고... 너무 힘들다ㅜ
P01: 머리 안 감고 출근해야겠다...
P02: 키키 그래 1학년인데. 아기상어 심했네
P01: 7살때 쓰던 거자나 굳이 학교선 필요 없으니 걍 넣어줬지
P01: 실컷 없이 잘 댕기다가 넣어달라 하자나... 또 사야지 모
P02: [사진]
P02: 돈 올린 거 봐 와 꼬람하네
P01: 헐 키키 어제 사고 잤제?
P02: 응 키키 샀지
P01: 다행이네 사고 자길 잘했구만
P02: 그니까 니 거는?
P01: 근데 손 닦는 거 손수건 보냄 안 되는가
P02: 난 그거 보냈는데?
P01: 내 건 인기 있는 게 아니라서 또 그림 애기 거라고 머라 칼라나
P01: 침 닦고 했던 그거 새 거 있거든
P02: 응 나도 그거 보냈는데 근래는 안 보냄
P01: 애기애기해 너
P01: 하니깐...또 싫다 칼 듯...


In [40]:
PROMPT_BASELINE = f"""아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

summary = summarize(
    conversation=get_eval_data()[10],
    prompt=PROMPT_BASELINE,
    model='claude-3-haiku-20240307'
)
print(summary)

요약:
1. P01과 P02는 아이디어스에서 돈을 벌고 있으며, 엄마와 조카의 계정도 사용하고 있다.
2. P01은 학교에 잠바를 갖다주러 가느라 힘들어하며, 손수건 문제로 고민하고 있다.
3. P02는 막창을 사고 돈을 벌었으며, P01에게 손수건을 보냈지만 P01은 만족스럽지 않은 것 같다.


In [42]:
summary = summarize(
    conversation=get_eval_data()[10],
    prompt=prompt,
    model='claude-3-haiku-20240307'
)
print(summary)

이 대화는 토스 아이디어스 앱과 관련된 이야기, 그리고 학교에 보내는 손수건 문제에 대해 다루고 있습니다. 주요 내용은 다음과 같습니다:

1. 토스 아이디어스 앱에서 돈을 벌고 있는 이야기와 함께 엄마와 신랑의 계정으로도 돈을 벌고 있다는 내용입니다. 
2. 학교에 보내는 손수건 문제로 인해 스트레스를 받고 있는 P01의 이야기가 나옵니다. 아기 상어 그림이 있는 손수건을 보내야 해서 부끄러워하고 있습니다.
3. 마지막으로 P02가 보낸 사진을 보며 돈이 올랐다는 내용으로 대화가 마무리됩니다.
